In [3]:
import pandas as pd

df = pd.read_csv('FFD.csv', sep = ",")
df.head()

,Unnamed: 0,cvr_base_data_table.conversion_status,cvr_base_data_table.ad_id,cvr_base_data_table.click_id,cvr_base_data_table.cc_id,cvr_base_data_table.hour_of_day,cvr_base_data_table.raw_publisher_id,cvr_base_data_table.raw_sub_publisher_id,cvr_base_data_table.raw_sub_sub_publisher_id,cvr_base_data_table.enriched_country,...,cvr_base_data_table.category_name,cvr_base_data_table.click_browser,cvr_base_data_table.click_user_datetime,cvr_base_data_table.raw_sync_partner_id,cvr_base_data_table.click_conf_gmt_offset,cvr_base_data_table.click_state,cvr_base_data_table.click_city,cvr_base_data_table.ad_bid,cvr_base_data_table.ad_cpa_goal,cvr_base_data_table.partner_id
0,0,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,429821366,us,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
1,1,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,429821366,us,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
2,2,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,8PO1W66UO,110622874,us,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:11:55.723,1,-600,al,birmingham,0.150000,NaN,8PRHGG6T9
3,3,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,8PO1W66UO,110622874,us,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 23:16:16.033,1,-500,in,ft wayne,0.150000,NaN,8PRHGG6T9
4,4,0,128699,246754980029002839,646754981260002933,22,8CUX8874N,8PO4X3YG2,670653476,us,...,"finance,grants,,scholarships,,,financial,aid","google,chrome",2019-01-05 22:09:41.026,1,-800,ca,mountain view,3.394224,NaN,8PRHGG6T9


In [4]:
for d in df.columns:
    if(df[d].nunique() == 1):
        print (d)

In [13]:
df.drop(['cvr_base_data_table.cc_id_null_ind', 'cvr_base_data_table.serving_key', 'cvr_base_data_table.ad_partner_id', 'cvr_base_data_table.ad_market_place_id', 'cvr_base_data_table.click_click_status', 'cvr_base_data_table.kmean_category_name', 'cvr_base_data_table.version'],axis = 1)

,cvr_base_data_table.ts,cvr_base_data_table.impression_system_date,cvr_base_data_table.hour,cvr_base_data_table.conversion_status,cvr_base_data_table.ad_id,cvr_base_data_table.click_id,cvr_base_data_table.cc_id,cvr_base_data_table.hour_of_day,cvr_base_data_table.raw_publisher_id,cvr_base_data_table.raw_sub_publisher_id,...,cvr_base_data_table.category_name,cvr_base_data_table.click_browser,cvr_base_data_table.click_user_datetime,cvr_base_data_table.raw_sync_partner_id,cvr_base_data_table.click_conf_gmt_offset,cvr_base_data_table.click_state,cvr_base_data_table.click_city,cvr_base_data_table.ad_bid,cvr_base_data_table.ad_cpa_goal,cvr_base_data_table.partner_id
0,20190106,2019-01-06,1,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
1,20190106,2019-01-06,1,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
2,20190106,2019-01-06,2,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,8PO1W66UO,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:11:55.723,1,-600,al,birmingham,0.150000,NaN,8PRHGG6T9
3,20190106,2019-01-06,4,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,8PO1W66UO,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 23:16:16.033,1,-500,in,ft wayne,0.150000,NaN,8PRHGG6T9
4,20190106,2019-01-06,6,0,128699,246754980029002839,646754981260002933,22,8CUX8874N,8PO4X3YG2,...,"finance,grants,,scholarships,,,financial,aid","google,chrome",2019-01-05 22:09:41.026,1,-800,ca,mountain view,3.394224,NaN,8PRHGG6T9
5,20190106,2019-01-06,6,0,150368,346756901408009927,546756902206003613,22,8CUX8874N,8PO4X3YG2,...,"finance,insurance","google,chrome",2019-01-05 22:41:42.207,1,-800,ca,mountain view,0.830000,10.0,8PRHGG6T9
6,20190106,2019-01-06,7,0,138155,146759863823002600,446759863997003939,23,8CUDMGV55,8PO1W66UO,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 23:31:03.997,1,-800,ca,modesto,0.529265,NaN,8PRHGG6T9
7,20190106,2019-01-06,13,1,138155,546780612913003471,646780613334006319,7,8CUDMGV55,8PO1W66UO,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-06 07:16:53.333,1,-600,tx,grand prairie,0.323027,NaN,8PRHGG6T9
8,20190106,2019-01-06,13,0,138155,146781810785000121,346781811230007878,8,8CUDMGV55,8PO1W66UO,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-06 08:36:51.023,1,-500,in,carmel,0.150000,NaN,8PRHGG6T9
9,20190106,2019-01-06,14,0,150711,646785644467002752,346785644736002846,8,8CU2G02TZ,8PO5Y867J,...,"finance,credit,,,lending,credit,cards","mozilla,firefox",2019-01-06 08:40:44.737,1,-600,tx,dallas,0.282782,40.0,8PRN625DH


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124335 entries, 0 to 124334
Data columns (total 42 columns):
cvr_base_data_table.ts                            124335 non-null int64
cvr_base_data_table.impression_system_date        124335 non-null object
cvr_base_data_table.hour                          124335 non-null int64
cvr_base_data_table.conversion_status             124335 non-null int64
cvr_base_data_table.cc_id_null_ind                124335 non-null int64
cvr_base_data_table.ad_id                         124335 non-null int64
cvr_base_data_table.click_id                      124335 non-null int64
cvr_base_data_table.cc_id                         124335 non-null int64
cvr_base_data_table.hour_of_day                   124335 non-null int64
cvr_base_data_table.raw_publisher_id              124335 non-null object
cvr_base_data_table.raw_sub_publisher_id          124335 non-null object
cvr_base_data_table.raw_sub_sub_publisher_id      124335 non-null object
cvr_base_data_table.e

In [15]:
df.head()

,cvr_base_data_table.ts,cvr_base_data_table.impression_system_date,cvr_base_data_table.hour,cvr_base_data_table.conversion_status,cvr_base_data_table.cc_id_null_ind,cvr_base_data_table.ad_id,cvr_base_data_table.click_id,cvr_base_data_table.cc_id,cvr_base_data_table.hour_of_day,cvr_base_data_table.raw_publisher_id,...,cvr_base_data_table.click_conf_gmt_offset,cvr_base_data_table.click_state,cvr_base_data_table.click_city,cvr_base_data_table.ad_bid,cvr_base_data_table.ad_cpa_goal,cvr_base_data_table.ad_market_place_id,cvr_base_data_table.click_click_status,cvr_base_data_table.kmean_category_name,cvr_base_data_table.partner_id,cvr_base_data_table.version
0,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
1,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
2,20190106,2019-01-06,2,0,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,...,-600,al,birmingham,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
3,20190106,2019-01-06,4,0,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,...,-500,in,ft wayne,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
4,20190106,2019-01-06,6,0,0,128699,246754980029002839,646754981260002933,22,8CUX8874N,...,-800,ca,mountain view,3.394224,NaN,1027,1,missing,8PRHGG6T9,1


In [8]:
df.columns

Index(['Unnamed: 0', 'cvr_base_data_table.conversion_status',
       'cvr_base_data_table.ad_id', 'cvr_base_data_table.click_id',
       'cvr_base_data_table.cc_id', 'cvr_base_data_table.hour_of_day',
       'cvr_base_data_table.raw_publisher_id',
       'cvr_base_data_table.raw_sub_publisher_id',
       'cvr_base_data_table.raw_sub_sub_publisher_id',
       'cvr_base_data_table.enriched_country',
       'cvr_base_data_table.enriched_derived_device',
       'cvr_base_data_table.enriched_derived_os',
       'cvr_base_data_table.enriched_publisher_domain',
       'cvr_base_data_table.ad_adgroup_id',
       'cvr_base_data_table.ad_advertiser_id',
       'cvr_base_data_table.ad_campaign_id', 'cvr_base_data_table.ad_keyword',
       'cvr_base_data_table.impression_state',
       'cvr_base_data_table.impression_city',
       'cvr_base_data_table.impression_isp_name',
       'cvr_base_data_table.impression_asn_code',
       'cvr_base_data_table.impression_connection_type',
       'cvr_base_da

In [5]:
df = df.drop(['cvr_base_data_table.cc_id_null_ind', 'cvr_base_data_table.serving_key', 'cvr_base_data_table.ad_partner_id', 'cvr_base_data_table.ad_market_place_id', 'cvr_base_data_table.click_click_status', 'cvr_base_data_table.kmean_category_name', 'cvr_base_data_table.version'],axis = 1)

KeyError: "['cvr_base_data_table.cc_id_null_ind' 'cvr_base_data_table.serving_key'\n 'cvr_base_data_table.ad_partner_id'\n 'cvr_base_data_table.ad_market_place_id'\n 'cvr_base_data_table.click_click_status'\n 'cvr_base_data_table.kmean_category_name' 'cvr_base_data_table.version'] not found in axis"

In [6]:
df = df.drop(['cvr_base_data_table.ts', 'cvr_base_data_table.impression_system_date', 'cvr_base_data_table.hour'], axis = 1)

KeyError: "['cvr_base_data_table.ts' 'cvr_base_data_table.impression_system_date'\n 'cvr_base_data_table.hour'] not found in axis"

In [11]:
df = df.drop('Unnamed: 0', axis = 1)
df.head()

,cvr_base_data_table.conversion_status,cvr_base_data_table.ad_id,cvr_base_data_table.click_id,cvr_base_data_table.cc_id,cvr_base_data_table.hour_of_day,cvr_base_data_table.raw_publisher_id,cvr_base_data_table.raw_sub_publisher_id,cvr_base_data_table.raw_sub_sub_publisher_id,cvr_base_data_table.enriched_country,cvr_base_data_table.enriched_derived_device,...,cvr_base_data_table.category_name,cvr_base_data_table.click_browser,cvr_base_data_table.click_user_datetime,cvr_base_data_table.raw_sync_partner_id,cvr_base_data_table.click_conf_gmt_offset,cvr_base_data_table.click_state,cvr_base_data_table.click_city,cvr_base_data_table.ad_bid,cvr_base_data_table.ad_cpa_goal,cvr_base_data_table.partner_id
0,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,429821366,us,mobile,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
1,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,429821366,us,mobile,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
2,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,8PO1W66UO,110622874,us,mobile,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:11:55.723,1,-600,al,birmingham,0.150000,NaN,8PRHGG6T9
3,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,8PO1W66UO,110622874,us,mobile,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 23:16:16.033,1,-500,in,ft wayne,0.150000,NaN,8PRHGG6T9
4,0,128699,246754980029002839,646754981260002933,22,8CUX8874N,8PO4X3YG2,670653476,us,mobile,...,"finance,grants,,scholarships,,,financial,aid","google,chrome",2019-01-05 22:09:41.026,1,-800,ca,mountain view,3.394224,NaN,8PRHGG6T9


In [7]:
for index, row in df.iterrows():
    if(row['cvr_base_data_table.enriched_country'] == 'au'):
        print (row)
        print ('\n')

Unnamed: 0                                                                                     7666
cvr_base_data_table.conversion_status                                                             0
cvr_base_data_table.ad_id                                                                    150993
cvr_base_data_table.click_id                                                    8547118931196001234
cvr_base_data_table.cc_id                                                       7347118931744005009
cvr_base_data_table.hour_of_day                                                                  22
cvr_base_data_table.raw_publisher_id                                                      8CUIOJ3V1
cvr_base_data_table.raw_sub_publisher_id                                                  8PO2C6731
cvr_base_data_table.raw_sub_sub_publisher_id                                              648516525
cvr_base_data_table.enriched_country                                                             au


In [22]:
print (df['cvr_base_data_table.click_conf_gmt_offset'].value_counts())

-500     60494
-600     34897
-800     20406
-700      7738
-1000      523
-900       271
 1100        3
 9999        3
Name: cvr_base_data_table.click_conf_gmt_offset, dtype: int64


In [43]:
import array
tot = array.array('i',(0 for i in range(0,24)))
converted = array.array('i',(0 for i in range(0,24)))

for index, row in df.iterrows():
    tot[row['cvr_base_data_table.hour_of_day']] = tot[row['cvr_base_data_table.hour_of_day']] + 1
    if(row['cvr_base_data_table.conversion_status'] == 1):
        converted[row['cvr_base_data_table.hour_of_day']] = converted[row['cvr_base_data_table.hour_of_day']] + 1
        
for hour in range (0,24):
    print (converted[hour], tot[hour], converted[hour] / tot[hour])

161 2733 0.05890962312477131
132 1980 0.06666666666666667
104 1516 0.06860158311345646
99 1422 0.06962025316455696
127 1592 0.07977386934673367
129 2114 0.061021759697256386
179 3119 0.05739018916319333
256 4897 0.0522769042270778
342 6718 0.05090800833581423
448 7682 0.058318146316063524
444 8178 0.05429200293470286
494 8346 0.059190031152647975
477 8516 0.05601221230624706
439 8338 0.052650515711201724
471 8354 0.05638017716064161
436 7672 0.05683003128258603
434 7173 0.06050467029137042
374 5846 0.06397536777283613
397 5660 0.07014134275618375
290 5185 0.055930568948891035
356 4822 0.07382828701783492
321 4845 0.06625386996904024
235 4285 0.05484247374562427
156 3342 0.0466786355475763


In [44]:
df['cvr_base_data_table.hour_of_day'].corr(df['cvr_base_data_table.conversion_status'])

0.002222302374722094

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124335 entries, 0 to 124334
Data columns (total 32 columns):
cvr_base_data_table.conversion_status             124335 non-null int64
cvr_base_data_table.ad_id                         124335 non-null int64
cvr_base_data_table.click_id                      124335 non-null int64
cvr_base_data_table.cc_id                         124335 non-null int64
cvr_base_data_table.hour_of_day                   124335 non-null int64
cvr_base_data_table.raw_publisher_id              124335 non-null object
cvr_base_data_table.raw_sub_publisher_id          124335 non-null object
cvr_base_data_table.raw_sub_sub_publisher_id      124335 non-null object
cvr_base_data_table.enriched_country              124335 non-null object
cvr_base_data_table.enriched_derived_device       124335 non-null object
cvr_base_data_table.enriched_derived_os           124335 non-null object
cvr_base_data_table.enriched_publisher_domain     124335 non-null object
cvr_base_data_tabl